In [42]:
#Importing libraries
import pandas as pd
from datetime import date, timedelta
import matplotlib.pyplot as plt
import numpy as np
import sdt.changepoint as c
import math

In [43]:
#import ARIMA libraries
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics import tsaplots
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
import numpy as np

In [44]:
# create a dataframe from the csv file
df_new_cases = pd.read_csv('../data/WHO-COVID-19-global-data.csv')
#df_vaccination = pd.read_csv('../data/owid-covid-data.csv')
df_vaccination = pd.read_csv('../data/country_vaccinations.csv')

In [45]:
# add moving average for new cases
#df_vaccination['moving_average_new_vaccinations'] = df_vaccination.groupby('iso_code')['new_vaccinations'].transform(lambda x: x.rolling(window=7).mean()) # OWID
df_vaccination['moving_average_new_vaccinations'] = df_vaccination.groupby('iso_code')['daily_vaccinations'].transform(lambda x: x.rolling(window=7).mean())

# rimuovi tutte le righe che non hanno un valore o hanno un valore a "0.0" per moving_average_new_cases
df_vaccination = df_vaccination.dropna(subset=['moving_average_new_vaccinations'])
df_vaccination = df_vaccination[df_vaccination['moving_average_new_vaccinations'] != 0.0]

In [46]:
# add moving average for new cases
df_new_cases['moving_average_new_cases'] = df_new_cases.groupby('Country_code')['New_cases'].transform(lambda x: x.rolling(window=7).mean())

# rimuovi tutte le righe che non hanno un valore o hanno un valore a "0.0" per moving_average_new_cases
df_new_cases = df_new_cases.dropna(subset=['moving_average_new_cases'])
df_new_cases = df_new_cases[df_new_cases['moving_average_new_cases'] != 0.0]

In [47]:
# firter data from 2021-07-16 to 2021-08-15
start_date = date(2021, 7, 1) 
og_start_date = date(2021, 7, 22)
og_end_date = date(2021, 8, 8)
end_date = date(2021, 8, 31)

difference = og_start_date - start_date
event_duration = og_end_date - og_start_date

# filter data from 2021-07-16 to 2021-08-15
df_new_cases = df_new_cases[(df_new_cases['Date_reported'] >= str(start_date)) & (df_new_cases['Date_reported'] <= str(end_date))]

# filter df_vaccination from 2021-07-16 to 2021-08-15
df_vaccination = df_vaccination[(df_vaccination['date'] >= str(start_date)) & (df_vaccination['date'] <= str(end_date))]

In [48]:
#filter by iso_cod that not contain "OWID" prefix
df_vaccination = df_vaccination[~df_vaccination['iso_code'].str.contains("OWID")]

# aggragate by iso_code and date and sum total_cases, total_deaths, population, total_vaccinations, new_cases, mind positive_rate, max positive_rate,KEEP LOCATION 
#df_vaccination = df_vaccination.groupby(['iso_code', 'date']).agg({'people_vaccinated':'sum', 'new_vaccinations': 'sum', 'location': 'max', 'moving_average_new_vaccinations': 'sum'})
df_vaccination = df_vaccination.groupby(['iso_code', 'date']).agg({'total_vaccinations':'sum', 'daily_vaccinations': 'sum', 'country': 'max', 'moving_average_new_vaccinations': 'sum'})

In [49]:
# aggragate by iso_code and date and sum total_cases, total_deaths, population, total_vaccinations, new_cases, mind positive_rate, max positive_rate,KEEP LOCATION 
df_new_cases = df_new_cases.groupby(['Country_code', 'Date_reported']).agg({'Country': 'sum', 'WHO_region': 'sum', 'New_cases': 'sum', 'Cumulative_cases': 'sum', 'New_deaths': 'sum', 'Cumulative_deaths': 'sum', 'moving_average_new_cases': 'sum'})

In [76]:
#aggiungi alla riga attuale nella colonna iso_code il valore iso
convert_ISO_3166_2_to_1 = {
    'AF':'AFG',
    'AX':'ALA',
    'AL':'ALB',
    'DZ':'DZA',
    'AS':'ASM',
    'AD':'AND',
    'AO':'AGO',
    'AI':'AIA',
    'AQ':'ATA',
    'AG':'ATG',
    'AR':'ARG',
    'AM':'ARM',
    'AW':'ABW',
    'AU':'AUS',
    'AT':'AUT',
    'AZ':'AZE',
    'BS':'BHS',
    'BH':'BHR',
    'BD':'BGD',
    'BB':'BRB',
    'BY':'BLR',
    'BE':'BEL',
    'BZ':'BLZ',
    'BJ':'BEN',
    'BM':'BMU',
    'BT':'BTN',
    'BO':'BOL',
    'BA':'BIH',
    'BW':'BWA',
    'BV':'BVT',
    'BR':'BRA',
    'IO':'IOT',
    'BN':'BRN',
    'BG':'BGR',
    'BF':'BFA',
    'BI':'BDI',
    'KH':'KHM',
    'CM':'CMR',
    'CA':'CAN',
    'CV':'CPV',
    'KY':'CYM',
    'CF':'CAF',
    'TD':'TCD',
    'CL':'CHL',
    'CN':'CHN',
    'CX':'CXR',
    'CC':'CCK',
    'CO':'COL',
    'KM':'COM',
    'CG':'COG',
    'CD':'COD',
    'CK':'COK',
    'CR':'CRI',
    'CI':'CIV',
    'HR':'HRV',
    'CU':'CUB',
    'CY':'CYP',
    'CZ':'CZE',
    'DK':'DNK',
    'DJ':'DJI',
    'DM':'DMA',
    'DO':'DOM',
    'EC':'ECU',
    'EG':'EGY',
    'SV':'SLV',
    'GQ':'GNQ',
    'ER':'ERI',
    'EE':'EST',
    'ET':'ETH',
    'FK':'FLK',
    'FO':'FRO',
    'FJ':'FJI',
    'FI':'FIN',
    'FR':'FRA',
    'GF':'GUF',
    'PF':'PYF',
    'TF':'ATF',
    'GA':'GAB',
    'GM':'GMB',
    'GE':'GEO',
    'DE':'DEU',
    'GH':'GHA',
    'GI':'GIB',
    'GR':'GRC',
    'GL':'GRL',
    'GD':'GRD',
    'GP':'GLP',
    'GU':'GUM',
    'GT':'GTM',
    'GG':'GGY',
    'GN':'GIN',
    'GW':'GNB',
    'GY':'GUY',
    'HT':'HTI',
    'HM':'HMD',
    'VA':'VAT',
    'HN':'HND',
    'HK':'HKG',
    'HU':'HUN',
    'IS':'ISL',
    'IN':'IND',
    'ID':'IDN',
    'IR':'IRN',
    'IQ':'IRQ',
    'IE':'IRL',
    'IM':'IMN',
    'IL':'ISR',
    'IT':'ITA',
    'JM':'JAM',
    'JP':'JPN',
    'JE':'JEY',
    'JO':'JOR',
    'KZ':'KAZ',
    'KE':'KEN',
    'KI':'KIR',
    'KP':'PRK',
    'KR':'KOR',
    'KW':'KWT',
    'KG':'KGZ',
    'LA':'LAO',
    'LV':'LVA',
    'LB':'LBN',
    'LS':'LSO',
    'LR':'LBR',
    'LY':'LBY',
    'LI':'LIE',
    'LT':'LTU',
    'LU':'LUX',
    'MO':'MAC',
    'MK':'MKD',
    'MG':'MDG',
    'MW':'MWI',
    'MY':'MYS',
    'MV':'MDV',
    'ML':'MLI',
    'MT':'MLT',
    'MH':'MHL',
    'MQ':'MTQ',
    'MR':'MRT',
    'MU':'MUS',
    'YT':'MYT',
    'MX':'MEX',
    'FM':'FSM',
    'MD':'MDA',
    'MC':'MCO',
    'MN':'MNG',
    'ME':'MNE',
    'MS':'MSR',
    'MA':'MAR',
    'MZ':'MOZ',
    'MM':'MMR',
    'NA':'NAM',
    'NR':'NRU',
    'NP':'NPL',
    'NL':'NLD',
    'AN':'ANT',
    'NC':'NCL',
    'NZ':'NZL',
    'NI':'NIC',
    'NE':'NER',
    'NG':'NGA',
    'NU':'NIU',
    'NF':'NFK',
    'MP':'MNP',
    'NO':'NOR',
    'OM':'OMN',
    'PK':'PAK',
    'PW':'PLW',
    'PS':'PSE',
    'PA':'PAN',
    'PG':'PNG',
    'PY':'PRY',
    'PE':'PER',
    'PH':'PHL',
    'PN':'PCN',
    'PL':'POL',
    'PT':'PRT',
    'PR':'PRI',
    'QA':'QAT',
    'RE':'REU',
    'RO':'ROU',
    'RU':'RUS',
    'RW':'RWA',
    'BL':'BLM',
    'SH':'SHN',
    'KN':'KNA',
    'LC':'LCA',
    'MF':'MAF',
    'PM':'SPM',
    'VC':'VCT',
    'WS':'WSM',
    'SM':'SMR',
    'ST':'STP',
    'SA':'SAU',
    'SN':'SEN',
    'RS':'SRB',
    'SC':'SYC',
    'SL':'SLE',
    'SG':'SGP',
    'SK':'SVK',
    'SI':'SVN',
    'SB':'SLB',
    'SO':'SOM',
    'ZA':'ZAF',
    'GS':'SGS',
    'ES':'ESP',
    'LK':'LKA',
    'SD':'SDN',
    'SR':'SUR',
    'SJ':'SJM',
    'SZ':'SWZ',
    'SE':'SWE',
    'CH':'CHE',
    'SY':'SYR',
    'TW':'TWN',
    'TJ':'TJK',
    'TZ':'TZA',
    'TH':'THA',
    'TL':'TLS',
    'TG':'TGO',
    'TK':'TKL',
    'TO':'TON',
    'TT':'TTO',
    'TN':'TUN',
    'TR':'TUR',
    'TM':'TKM',
    'TC':'TCA',
    'TV':'TUV',
    'UG':'UGA',
    'UA':'UKR',
    'AE':'ARE',
    'GB':'GBR',
    'US':'USA',
    'UM':'UMI',
    'UY':'URY',
    'UZ':'UZB',
    'VU':'VUT',
    'VE':'VEN',
    'VN':'VNM',
    'VG':'VGB',
    'VI':'VIR',
    'WF':'WLF',
    'EH':'ESH',
    'YE':'YEM',
    'ZM':'ZMB',
    'ZW':'ZWE'
}

for code in df_new_cases.index.get_level_values('Country_code').unique():
    if code not in country_code_to_iso:
        print(code , "NOT FOUND")
        continue
    iso = country_code_to_iso[code]
    df_new_cases.loc[(df_new_cases.index.get_level_values('Country_code') == code), 'iso_code'] = iso

In [71]:
# save dataframe in csv file
df_new_cases.to_csv('./dataframe.csv')
df_vaccination.to_csv('./vaccinations.csv')

In [72]:
'''for iso_code in df_vaccination.index.get_level_values('iso_code').unique():
    plt.plot(df_vaccination.loc[iso_code]['moving_average_new_vaccinations'], label='vaccinations')
    plt.title(df_vaccination.loc[iso_code]['country'].unique()[0])
    plt.ylabel("new vaccinations")
    plt.xlabel("Date")
    plt.xticks(rotation=90)
    plt.show()'''

'for iso_code in df_vaccination.index.get_level_values(\'iso_code\').unique():\n    plt.plot(df_vaccination.loc[iso_code][\'moving_average_new_vaccinations\'], label=\'vaccinations\')\n    plt.title(df_vaccination.loc[iso_code][\'country\'].unique()[0])\n    plt.ylabel("new vaccinations")\n    plt.xlabel("Date")\n    plt.xticks(rotation=90)\n    plt.show()'

In [83]:

# crea una lista di tutti gli iso_code presenti in df_new_cases
iso_code_list = df_new_cases['iso_code'].unique().tolist()
#print(len(iso_code_list))
#print(df_new_cases.index.levels[0].shape[0])

In [ ]:
import datetime
#interval technique
BayesOffline = c.BayesOffline()
media_changepoint = 0.0
count_changepoint = 0.0
min_changepoint = (999,"error")
max_changepoint = (-1,"error")
save_me = []
tabella = pd.DataFrame(columns=['State','FirstSlope','SecondSlope','ThirdSlope','FourthSlope'])
trend_negative = []
trend_positive = []

for code in all_iso_codes:
    # Out is a list of possible changepoint "indices"
    values = df_new_cases.loc[code]['moving_average_new_cases'].values
    
    #crea un df 
    df_cases = pd.DataFrame(values, index = df_new_cases.loc[code]['moving_average_new_cases'].index, columns = ['new_cases'])
    df_vaccin = pd.DataFrame(df_vaccination.loc[code]['moving_average_new_vaccinations'].values, index = df_vaccination.loc[code]['moving_average_new_vaccinations'].index, columns = ['new_vaccinations'])
    
    merged_df = df_cases.merge(df_vaccination, on=['iso_code', 'date'])
    print(merged_df)
    print(merged_df.index)
    
    
    
    # crea un df con i valori di new_cases e indice le righe con le date
    df = pd.DataFrame(values, index = df_vaccination.loc[code]['moving_average_new_vaccinations'].index, columns = ['new_cases'])
    print(df)
    print(df.index)
    
    #splitta il df in due dataframe di training e di test con split date
    df_train = df.loc[df.index <= str(split_date)].copy()
    df_test = df.loc[df.index > str(split_date)].copy()
    
    
    # crea un modello ARIMA per predirre i new_cases considerando i vaccini fino 2 settimane prima dell'inizio delle olimpiadi
    ARIMAModel = ARIMA(df_train, order=(1,1,1))
    ARIMAModel_fit = ARIMAModel.fit()
    ARIMAModel_fit.summary()
    # predice i new_cases per le due settimane delle olimpiadi
    ARIMAModel_fit.predict(start = len(df_train), end = len(df_train)+13)
    # calcola la media dei new_cases predetti
    mean_predicted = ARIMAModel_fit.predict(start = len(df_train), end = len(df_train)+13).mean()
    # calcola la media dei new_cases reali
    mean_real = df_test.mean()
    # calcola la differenza tra la media dei new_cases predetti e la media dei new_cases reali
    difference = mean_predicted - mean_real
    # calcola la percentuale di differenza tra la media dei new_cases predetti e la media dei new_cases reali
    percentage_difference = (difference/mean_real)*100
    print("The mean of the predicted new cases is: " + str(mean_predicted))
    
    # plot the predicted values
    plt.plot(ARIMAModel_fit.predict(start = len(df_train), end = len(df_train)+13), color = 'red', label = 'Predicted new cases')
    # plot the real values
    plt.plot(df_test, color = 'blue', label = 'Real new cases')
    plt.title(code + " - " + df_new_cases.loc[code]['Country'].unique()[0])
    plt.xlabel("Days since the 28th of July 2021")
    plt.ylabel("Current MA - Previous MA")
    plt.legend()
    plt.show()
    
    
    '''df2 = df_new_cases.loc[code]['moving_average_new_cases']
    # Questa riga estrae i valori della colonna "Data" del dataframe df dalla prima data del dataframe alla data del lockdown.
    values_before = df2.iloc[:difference.days+1].values
    # Questa riga estrae i valori della colonna "Data" del dataframe df dalla data del lockdown fino alla settimana successiva 4.
    values_oneWeek = df2.iloc[difference.days+1:difference.days+8].values
    # Questa riga estrae i valori della colonna "Data" del dataframe df dalla data del lockdown fino alla settimana successiva 5.
    values_twoWeek = df2.iloc[difference.days+8:difference.days+15].values
    # Questa riga estrae i valori della colonna "Data" del dataframe df dalla data del lockdown fino alla settimana successiva 6.
    values_after = df2.iloc[difference.days+15:].values


    z1 = np.polyfit(range(0,len(values_before)), values_before, 1)
    p1 = np.poly1d(z1)
    z2 = np.polyfit(range(len(values_before),len(values_before)+len(values_oneWeek)), values_oneWeek, 1)
    p2 = np.poly1d(z2)
    z3 = np.polyfit(range(len(values_before)+len(values_oneWeek),len(values_before)+len(values_oneWeek)+len(values_twoWeek)), values_twoWeek, 1)
    p3 = np.poly1d(z3)
    z4 = np.polyfit(range(len(values_before)+len(values_oneWeek)+len(values_twoWeek),len(values_before)+len(values_oneWeek)+len(values_twoWeek)+len(values_after)), values_after, 1)
    p4 = np.poly1d(z4)
    
    entry = pd.DataFrame.from_dict({"State": code,'FirstSlope': [z1[0]],'SecondSlope': [z2[0]],'ThirdSlope': [z3[0]],'FourthSlope': [z4[0]]})
    tabella = pd.concat([tabella2, entry], ignore_index=True)

    array1 = np.arange(0,len(values_before))
    array2 = np.arange(len(values_before),len(values_before)+len(values_oneWeek))
    array2 = np.insert(array2, 0, array1[-1])
    array3 = np.arange(len(values_before)+len(values_oneWeek),len(values_before)+len(values_oneWeek)+len(values_twoWeek))
    array3 = np.insert(array3, 0, array2[-1])
    array4 = np.arange(len(values_before)+len(values_oneWeek)+len(values_twoWeek),len(values_before)+len(values_oneWeek)+len(values_twoWeek)+len(values_after))
    array4 = np.insert(array4, 0, array3[-1])
    lw = 2
    

    plt.plot(array1, p1(range(0,len(values_before))), color="#0B1F65", linewidth=lw, linestyle="dashdot")
    plt.plot(array2, p2(range(len(values_before)-1,len(values_before)+len(values_oneWeek))), color="#116530", linewidth=lw, linestyle="dashdot")
    plt.plot(array3, p3(range(len(values_before)+len(values_oneWeek)-1,len(values_before)+len(values_oneWeek)+len(values_twoWeek))), color="#D7A449", linewidth=lw, linestyle="dashdot")
    plt.plot(array4, p4(range(len(values_before)+len(values_oneWeek)+len(values_twoWeek)-1,len(values_before)+len(values_oneWeek)+len(values_twoWeek)+len(values_after))), color="#DB3F29", linewidth=lw, linestyle="dashdot")
    '''
    # Crea un plot con i punti di cambio colorati
    plt.title(code + " - " + df_new_cases.loc[code]['Country'].unique()[0])
    plt.axvline(x = difference.days-14, color = 'blue', label = '14 days before first game of olympic games')
    plt.axvline(x = difference.days+7, color = 'orange', label = '7 days after first game of olympic games')
    plt.axvline(x = difference.days+18, color = '#FF7D4D', label = '7 days after last game of olympic games')
    plt.xlabel("Days since the 28th of July 2021")
    plt.ylabel("Current MA - Previous MA")
    #plt.legend()
    plt.axvspan(difference.days, difference.days+11, facecolor='#ffe206', alpha=0.5)
    ymax = plt.ylim()[1]
    # aggiungi un margine di 5% sopra il massimo valore per far spazio alle etichette
    plt.ylim(ymax=ymax*1.20)
    plt.text(difference.days-14, ymax, "14 days before\nfirst game of\nolympic games", rotation=0, color="white", backgroundcolor="blue")
    plt.text(difference.days, ymax, "Olympic \nGames", rotation=0, color="black", backgroundcolor="yellow")
    plt.text(difference.days+7+1, ymax, "7 days \nafter first metch", rotation=0, backgroundcolor="orange")
    plt.text(difference.days+18+1, ymax, "7 days \nafter last match", rotation=0, backgroundcolor="#FF7D4D")
    #save the plot as a png file
    plt.savefig("../pictures/"+ code + "_intervals" +".png")
    plt.show()

In [ ]:
#tabella is the table for changepoints and tabella2 is the table for the intervals
tabella.set_index(tabella['State'])
tabella.to_csv("./tabella.csv", index=False)

tabella2.set_index(tabella2['State'])
tabella2.to_csv("./tabella2.csv", index=False)